In [0]:
!pip install openai==0.28

In [0]:
dbutils.library.restartPython()

In [0]:
import openai
from pathlib import Path
import os
import pandas as pd
from ParsingMemoriaCalculo import tabela_texto_memoria_para_df
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2023-12-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea','azure-oai-dk')

In [0]:
client = openai.OpenAI(api_key=dbutils.secrets.get('akvdesafiokinea','azure-oai-dk'))
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)

In [0]:
def posicoes(fundo,ativos):
    #Resgatar Régua Completa
    regua = pd.read_csv("/Volumes/desafio_kinea/boletagem_cp/files/InputNexxus/input_nexxus.csv",sep=',').drop('Unnamed: 0', axis=1)
    print(regua)
    posicoes = regua[(regua['fundo'] == fundo)  & (regua['ativo'].isin(ativos))].index
    return posicoes


In [0]:
#Função para Gerar Restrição
def gerar_restricao(descricao,ativos,saldo_base,exposicao,l_max,l_min):   
    contexto_gpt = '''
        Você irá receber a descrição de uma regra, um conjunto posições de ativos que são considerados para o cálculo dessa regra, um saldo base, limite minimo e máximo em relação a esse saldo base e a porcentagem que não por essa restrição. Lembre-se de considerar se a porcentagem está acima do limite_maximo ou abaixo do limite minimo para tomar a decisão de como escrever o código
        A partir disso você deve gerar um código de restrição no formato utilizado pelo scipy.minimize.
        
        Exemplo 1:
            Descrição: Art. 45. Cumulativamente aos limites de concentração por emissor, a classe de cotas deve observar os seguintes limites de concentração por modalidade de ativo financeiro, sem prejuízo das normas aplicáveis ao seu tipo: I - ate 20% (vinte por cento) de ativos de renda fixa;
            Posição Ativos: [0,1,4,5,6]
            Saldo Base: 500000
            Limite Mínimo: 0
            Limite Máximo: 0.2
            Porcentagem:0.25
            def restricao(x):
                ativos_considerados = [x[0],x[1],x[4],x[5],x[6]...]
                return  0.2 * 500000 -  sum(ativos_considerados)
        Exemplo 2: 
            Descrição: Regulamento - POLÍTICA DE INVESTIMENTO - O objetivo do FUNDO é aplicar, no mínimo, 80% (oitenta por cento) de seus recursos em ativos financeiros de renda fixa relacionados diretamente, ou sintetizados via derivativos, ao fator de risco que dá nome à classe, observado que a rentabilidade do FUNDO será impactada pelos custos e despesas do FUNDO, inclusive taxa de administração.
            Posição Ativos: [0,1,4,5,6]
            Saldo Base: 500000
            Limite Mínimo: 0.8
            Limite Máximo: 1
            Porcentagem: 0.68
            def restricao(x):
                ativos_considerados = [x[0],x[1],x[4],x[5],x[6]...]
                return sum(ativos_considerados) - 0.8 * 500000    
    '''
    message_text = [
                {"role":"system","content":contexto_gpt},
                {"role":"user","content":f"Descrição: {descricao}\n Posições Ativos: {ativos} \n Saldo Base: {saldo_base} \n Limite Mínimo {l_min}  \n Limite Máximo: {l_max} \n Porcentagem: {exposicao}"}
               ]
    
    completion = openai.ChatCompletion.create(
        engine="gpt35turbo16k",
        messages = message_text,
        temperature=0.0,
        max_tokens=1200,
        top_p=0.95,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=None,
        )

    return completion.to_dict()['choices'][0]['message']['content']



In [0]:
#Testando Função Regra L636
df_teste = spark.sql("select IdRegra, SaldoBaseCalculo,SaldoObjeto,LimiteMin,LimiteMax,ValorExposicao from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where IdFundo = 'CP4' and IdRegra = 'L636' and DataHoraVersao = '2025-02-07T11:22:56.000+00:00'").toPandas()

df_teste['DescricaoDetalhada'] = spark.sql("select DescricaoDetalhada from desafio_kinea.boletagem_cp.nxenq_regras where IdRegra = 636").toPandas()['DescricaoDetalhada'][0]

posicao_ativos = [0,2,5,8,9] #Iremos conseguir a partir da memória de cálculo 


resposta = gerar_restricao(df_teste['DescricaoDetalhada'][0],posicao_ativos,df_teste['SaldoBaseCalculo'][0],df_teste['LimiteMin'][0],df_teste['LimiteMax'][0],df_teste['ValorExposicao'][0])
print(resposta)

In [0]:
#Testando Função Regra L153
df_teste = spark.sql("select IdRegra, SaldoBaseCalculo,SaldoObjeto,LimiteMin,LimiteMax,ValorExposicao from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where IdFundo = 'CP8' and IdRegra = 'CD153' and status='1' and DataHoraVersao = '2025-02-10T11:12:37.000+00:00'").toPandas()

df_teste['DescricaoDetalhada'] = spark.sql("select DescricaoDetalhada from desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao where IdRegra = 153").toPandas()['DescricaoDetalhada'][0]

posicao_ativos = [2,6] #Iremos conseguir a partir do ComplementoID

resposta = gerar_restricao(df_teste['DescricaoDetalhada'][0],posicao_ativos,df_teste['SaldoBaseCalculo'][0],df_teste['LimiteMin'][0],df_teste['LimiteMax'][0],df_teste['ValorExposicao'][0])
print(resposta)

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_resultadoenquadramentohist where IdFundo = 'CP8' and IdRegra = 'CD153'  and DataHoraVersao = '2025-02-10T11:12:37.000+00:00' and status = 1

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_regrasporconcentracao 

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_regras

In [0]:
%sql
select * from desafio_kinea.boletagem_cp.nxenq_memoriacalculo where DataHoraVersao = '2025-02-10T11:12:37.000+00:00' and IdFundo = 'CP8' 